# Hyperparameter optimization
---
This notebook uses Cirrus to optimize the hyperparameters of a logistic regression model.

## Setup
---

In [ ]:
# To ease development, each time a cell is run, all modules will be reloaded.
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import sys
import atexit

from cirrus import instance, automate, lr, GridSearch

In [ ]:
# Cirrus produces logs, but they will not show unless we add a handler that prints.
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("[%(funcName)16s | %(threadName)15s] %(message)s")
handler.setFormatter(formatter)
logger = logging.getLogger("cirrus")
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)

## Instances, base task configuration, hyperparameters
---

First, we start some EC2 instances.

In [ ]:
NUM_INSTANCES = 2

instances = []
for i in range(NUM_INSTANCES):
    inst = instance.Instance(
        name="hyperparameter_example_instance_%d" % i,
        disk_size=32,
        typ="m5a.2xlarge",
        username="ubuntu",
        ami_owner_name=("self", "cirrus_server_image")
    )
    inst.start()
    instances.append(inst)

Second, we define the base configuration for our machine learning task.

In [ ]:
base_task_config = {
    "n_workers": 16,
    "n_ps": 1,
    "dataset": "criteo-kaggle-19b",
    "learning_rate": 0.0001,
    "epsilon": 0.0001,
    "progress_callback": None,
    "train_set": (0, 799),
    "test_set": (800, 915),
    "minibatch_size": 200,
    "model_bits": 19,
    "opt_method": "adagrad",
    "timeout": 60,
    "lambda_size": 192
}

Third, we identify our hyperparameters and their possible values.

In [ ]:
hyperparameter_names = [
    "n_workers",
    "learning_rate"
]
hyperparameter_values = [
    [8, 16, 32],
    [0.001, 0.01]
]

All of the above defines a hyperparameter optimization task, which consists of one machine learning task per assignment of values to the hyperparameters.

In [ ]:
search = GridSearch(
    task=lr.LogisticRegression,
    param_base=base_task_config,
    hyper_vars=hyperparameter_names,
    hyper_params=hyperparameter_values,
    instances=instances
)

## Run
---

Next, we run our hyperparameter optimization task.

In [ ]:
search.run()

Run this cell to see the present accuracy of experiment `I`.

In [ ]:
I = 0

for line in search.cirrus_objs[I].ps.error_output().split("\n")[-20:]:
    print(line)

## Cleanup
---

When we're satisfied with the results, we kill our task.

In [ ]:
search.kill_all()

We also need to terminate our instances in order to avoid continuing charges.

In [ ]:
for inst in instances:
    inst.cleanup()

If a cell errors, running this should clean up any resources that were created. After running this cell, the kernel will become unusable and need to be restarted.

In [ ]:
atexit._run_exitfuncs()